In [21]:
import pandas as pd
df = pd.read_csv('dataset-SA.csv')
df

,product_name,product_price,Rate,Review,Summary,Sentiment
0,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,super!,great cooler excellent air flow and for this p...,positive
1,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,awesome,best budget 2 fit cooler nice cooling,positive
2,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,the quality is good but the power of air is de...,positive
3,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,1,useless product,very bad product its a only a fan,negative
4,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,ok ok product,neutral
...,...,...,...,...,...,...
205044,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,5,must buy!,good product,positive
205045,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,5,super!,nice,positive
205046,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,3,nice,very nice and fast delivery,positive
205047,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,5,just wow!,awesome product,positive


In [22]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset (replace 'your_dataset.csv' with the actual dataset file)
df = pd.read_csv('Dataset-SA.csv')

# Preprocessing using spaCy
nlp = spacy.load('en_core_web_sm')

def text_data_cleaning(sentence):
    if isinstance(sentence, str):  # Check if 'sentence' is a string
        doc = nlp(sentence)
        tokens = [token.lemma_.lower().strip() if token.lemma_ != "-PRON-" else token.lower_ for token in doc]
        cleaned_tokens = [token for token in tokens if token.isalpha() and token not in nlp.Defaults.stop_words]
        return ' '.join(cleaned_tokens)
    else:
        return ''  # Handle non-string input by returning an empty string

# Apply text cleaning to the 'verified_reviews' column
df['Cleaned_Review'] = df['Summary'].apply(text_data_cleaning)

# Define a function to compute sentiments based on numeric labels
def compute_sentiments(labels):
    sentiments = []
    for label in labels:
        if label > 3.0:
            sentiment = 1
        elif label <= 3.0:
            sentiment = 0
        sentiments.append(sentiment)
    return sentiments

# Apply the compute_sentiments function to the 'rating' column
df['Sentiment'] = compute_sentiments(df['Rate'])

# Split the dataset into features (X) and labels (y)
X = df['Cleaned_Review'].tolist()  # Convert the column to a list of strings
y = df['Sentiment']  # Assuming 'Sentiment' is the column with sentiment labels (0 for negative, 1 for positive)

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # You can adjust the number of features as needed
X_tfidf = tfidf_vectorizer.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Train a Logistic Regression model
logistic_classifier = LogisticRegression()
logistic_classifier.fit(X_train, y_train)



c:\Users\rpate\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [27]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [29]:
# Predict using the trained Logistic Regression model
y_pred = logistic_classifier.predict(X_test)


In [30]:
# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Generate a classification report
classification_rep = classification_report(y_test, y_pred)
print('Classification Report:')
print(classification_rep)


Accuracy: 0.89
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.55      0.67      8760
           1       0.89      0.98      0.93     32250

    accuracy                           0.89     41010
   macro avg       0.87      0.76      0.80     41010
weighted avg       0.88      0.89      0.88     41010



In [19]:
# Make predictions
customer_review = "bad good good"
review_tfidf = tfidf_vectorizer.transform([customer_review])  # Vectorize the customer review using the same vectorizer
prediction = logistic_classifier.predict(review_tfidf)

# Interpret the prediction
sentiment = "negative" if prediction[0] == 0 else "positive"

# Print the prediction
print(f"Customer Review: {customer_review}")
print(f"Predicted Sentiment: {sentiment}")

Customer Review: bad good good
Predicted Sentiment: negative


In [31]:
import pickle

# Save the trained Logistic Regression model and TF-IDF vectorizer as pickle files
with open('logistic_regression_model.pkl', 'wb') as model_file:
    pickle.dump(logistic_classifier, model_file)

with open('tfidf_vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(tfidf_vectorizer, vectorizer_file)





In [33]:
# Sample review
sample_review = "worst product ever"

# Load the model and vectorizer from pickle files
with open('logistic_regression_model.pkl', 'rb') as model_file:
    loaded_logistic_classifier = pickle.load(model_file)

with open('tfidf_vectorizer.pkl', 'rb') as vectorizer_file:
    loaded_tfidf_vectorizer = pickle.load(vectorizer_file)

# Vectorize the sample review using the loaded vectorizer
sample_review_tfidf = loaded_tfidf_vectorizer.transform([sample_review])

# Make a prediction using the loaded model
prediction = loaded_logistic_classifier.predict(sample_review_tfidf)

# Interpret the prediction
sentiment = "negative" if prediction[0] == 0 else "positive"

# Print the prediction
print(f"Sample Review: {sample_review}")
print(f"Predicted Sentiment: {sentiment}")


Sample Review: worst product ever
Predicted Sentiment: negative
